# Test API

In [ ]:
import os
os.environ["planner_api"] = ""

In [ ]:
from core.llm_factory import LLM_factory
test_llm_model = LLM_factory.get_llm(agent_name="default")

In [3]:
test_llm_model.invoke("Lan đần ngố là đứa nào")

AIMessage(content='"Lan đần ngố" có vẻ là một cách nói đùa hoặc biệt danh mà bạn đang nhắc đến. Nếu bạn muốn nói về một người cụ thể, bạn có thể cung cấp thêm thông tin để mình hiểu rõ hơn. Tuy nhiên, hãy nhớ rằng việc sử dụng biệt danh hoặc từ ngữ có thể gây hiểu lầm hoặc không phù hợp, nên cân nhắc cách diễn đạt để tránh làm người khác cảm thấy khó chịu nhé! 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 16, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CsCouzT03h37hUQY3gzcy0a48JMaw', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--11df14ec-ec90-4935-8ca7-bca43d3d48da-0', usage_metadata={'input_tokens': 16, 'output_tokens

# Test function

In [2]:
import sys
import os
sys.path.append(os.getcwd())

In [1]:
from orchestrations.state import State
from langchain_core.messages import HumanMessage, AIMessage

initial_state: State = {
    "messages": [HumanMessage(content="Tìm kiếm thông tin về giá vàng hôm nay và vẽ biểu đồ.")],
    "user_query": "Tìm kiếm thông tin về giá vàng hôm nay và vẽ biểu đồ.",
    "enabled_agent": ["web_researcher", "chart_generator"],
    "plan": [],
    "current_step": 0,
    "agent_query": None,
    "last_reason": None,
    "replan_flag": False,
    "replan_attempts": {}
}

in_progress_state: State = {
    "messages": [
        HumanMessage(content="Tìm kiếm thông tin về giá vàng hôm nay..."),
        AIMessage(content='{"1": {"agent": "web_researcher", "goal": "Fetch gold price"}}')
    ],
    "user_query": "Tìm kiếm thông tin về giá vàng hôm nay...",
    "enabled_agents": ["web_researcher", "chart_generator"],
    "plan": [
        {
            1: {
                "agent": "web_researcher",
                "action": "search",
                "goal": "Lấy dữ liệu giá vàng mới nhất",
                "pre_conditions": [],
                "post_conditions": ["có dữ liệu giá"]
            }
        },
        {
            2: {
                "agent": "chart_generator",
                "action": "plot",
                "goal": "Vẽ biểu đồ xu hướng",
                "pre_conditions": ["có dữ liệu giá"],
                "post_conditions": ["file ảnh biểu đồ"]
            }
        }
    ],
    "current_step": 1,
    "agent_query": "Giá vàng thế giới hôm nay bao nhiêu?",
    "last_reason": "Cần dữ liệu thô trước khi vẽ biểu đồ.",
    "replan_flag": False,
    "replan_attempts": {1: 0}
}

In [ ]:
from core.llm_factory import LLM_factory
REASONING_LLM = LLM_factory.get_llm("planner")

/Users/nhatminhnguyen/anaconda3/envs/agents_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
from agents.planner import plan_prompt
test_plan_prompt = plan_prompt(state=in_progress_state)

In [23]:
test_plan_prompt

HumanMessage(content='You are the **Planner** in a multi‑agent system.  Break the user\'s request\ninto a sequence of numbered steps (1,\u202f2,\u202f3, …).  **There is no hard limit on\nstep count** as long as the plan is concise and each step has a clear goal.\n\nYou may decompose the user\'s query into sub-queries, each of which is a\nseparate step.  Break the query into the smallest possible sub-queries\nso that each sub-query is answerable with a single data source.\nFor example, if the user\'s query is "What were the key\naction items in the last quarter, and what was a recent news story for \neach of them?", you may break it into steps:\n\n1. Fetch the key action items in the last quarter.\n2. Fetch a recent news story for the first action item.\n3. Fetch a recent news story for the second action item.\n4. Fetch a recent news story for the last action item\n\nHere is a list of available agents you can call upon to execute the tasks in your plan. You may call only one agent per s

In [24]:
llm_reply = REASONING_LLM.invoke([plan_prompt(state=in_progress_state)])


In [25]:
llm_reply

AIMessage(content='{\n  "1": {\n    "agent": "web_researcher",\n    "action": "Search for today\'s gold prices in Vietnam.",\n    "goal": "Retrieve the current gold prices in Vietnam.",\n    "pre_conditions": [],\n    "post_conditions": ["Current gold prices in Vietnam are retrieved."]\n  }\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 487, 'total_tokens': 553, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CsbUfUkuhRmmiaLtiHY0a2A2QB3pE', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--b9f271dd-fbc6-4d3d-abf4-06a0f0549990-0', usage_metadata={'input_tokens': 487, 'output_tokens': 66, 'total_tokens': 553, 'input_token_details': {

In [26]:
print(llm_reply.content)

{
  "1": {
    "agent": "web_researcher",
    "action": "Search for today's gold prices in Vietnam.",
    "goal": "Retrieve the current gold prices in Vietnam.",
    "pre_conditions": [],
    "post_conditions": ["Current gold prices in Vietnam are retrieved."]
  }
}


# Test Something

In [2]:
from utils.helper import get_enabled_agents
type_executors = {"`, `".join(sorted(set([a for a in get_enabled_agents(initial_state) if a in ["web_researcher", "cortex_researcher", "chart_generator", "chart_summarizer", "synthesizer"]] + ["planner"])))}

In [3]:
type_executors

{'chart_generator`, `chart_summarizer`, `planner`, `synthesizer`, `web_researcher'}